# Get Votes

Récupérer les votes des parlementaires actuels en JSON grâce à l’API du parlement

http://ws-old.parlament.ch/

«Pour le moment, les nouveaux services web ne sont pas encore disponibles. Nous sommes cependant certains que nous pourrons remédier à cette situation d’ici au troisième trimestre 2019.»

Votes:
http://ws-old.parlament.ch/votes/councillors

Exemple de liste de votes:


http://ws-old.parlament.ch/votes/councillors/3055?legislativePeriodFiter=662


In [ ]:
import pandas as pd
import json
import requests
from time import sleep

pd.set_option("display.max_colwidth", 999)

In [ ]:
# df à actualiser avec Get_Councellors.ipynb
df_national = pd.read_csv("data/national.csv")
df_etats = pd.read_csv("data/states.csv")

In [ ]:
df_national["factionName"].value_counts()

factionName
Groupe de l'Union démocratique du Centre    55
Groupe socialiste                           39
Le Groupe du Centre. Le Centre. PEV.        31
Groupe des Verts                            30
Groupe libéral-radical                      29
Groupe vert'libéral                         16
Name: count, dtype: int64

In [ ]:
# Si on veut cibler un groupe précis:
# for i, row in df_national[df_national['factionName'] == 'Groupe PDC'].iterrows():

# On peut aussi filtrer avec legislativePeriodFilter=662

# Actuellement limité aux 5 premiers. Retirer "[:5]"
for i, row in df_national[:5].iterrows():
    print(i, row["firstName"], row["lastName"], row["id"], row["number"])
    number = row["number"]
    response = requests.get(
        "http://ws-old.parlament.ch/votes/councillors/"
        + str(number)
        + "?format=json&lang=fr",
        headers={
            "Accept": "application/json",
            "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
        },
    )
    try:
        df = pd.DataFrame(response.json())
    except:
        print("JSON invalide pour", row["firstName"], row["lastName"])
        continue

    # On aplatit le json
    affairs = df["affairVotes"].tolist()
    df_affair = pd.DataFrame(affairs)
    df_affair["decision"] = df_affair["councillorVote"].apply(lambda x: x["decision"])

    dfm = df.join(df_affair, rsuffix="_")
    del dfm["affairVotes"]
    del dfm["councillorVote"]

    # On enregistre
    # dfm.to_csv('data/votes_national/{}_votes.csv'.format(number))
    sleep(0.5)

0 Jean-Luc Addor 4154 3055
1 Andreas Aebi 3867 2670
2 Matthias Aebischer 4049 2760
3 Thomas Aeschi 4053 2758
4 Céline Amaudruz 4090 2796


In [ ]:
# Idem pour les Etats

# Actuellement limité aux 5 premiers. Retirer "[:5]" pour enlever cette limite
for i, row in df_etats[:5].iterrows():
    print(i, row["firstName"], row["lastName"], row["id"], row["number"])
    number = row["number"]
    response = requests.get(
        "http://ws-old.parlament.ch/votes/councillors/{}?format=json&lang=fr".format(
            number
        ),
        headers={
            "Accept": "application/json",
            "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
        },
    )
    try:
        df = pd.DataFrame(response.json())
    except:
        print("JSON invalide pour", row["firstName"], row["lastName"])
        continue

    affairs = df["affairVotes"].tolist()
    df_affair = pd.DataFrame(affairs)

    # On aplatit et enregistre
    dfm = df.join(df_affair, rsuffix="_")
    del dfm["affairVotes"]
    del dfm["councillorVote"]
    dfm.to_csv("data/votes_etats/{}_votes.csv".format(number))
    sleep(0.5)

0 Philippe Bauer 4187 3059
1 Elisabeth Baume-Schneider 4238 3193
JSON invalide pour Elisabeth Baume-Schneider
2 Pirmin Bischof 3871 2674
3 Thierry Burkart 4189 3065
4 Marina Carobbio Guscetti 3830 2666


And voilà!


# Aller plus loin

On peut regrouper tous ces fichiers CSV de votes…

Le module **glob** nous permettra d’avaler tous les fichiers CSV d’un coup. Si Python vous dit que vous ne l’avez pas, vous m’appelez.


In [6]:
from glob import glob

In [ ]:
df = pd.concat(map(pd.read_csv, glob("data/votes_national/*.csv")))
df.shape

(250, 17)

In [ ]:
del df["Unnamed: 0"]
df.index = pd.to_datetime(df["date"])

In [ ]:
df_national_simple = df_national[["number", "party"]]

In [ ]:
dfm = df.merge(df_national_simple, left_on="id", right_on="number")

In [11]:
# yeah!
dfm.head()

,id,updated,elanId,firstName,lastName,id_,affairId,affairTitle,date,divisionText,meaningNo,meaningYes,registrationNumber,submissionText,hasMorePages,decision,number,party
0,2796,2020-09-04T05:45:44Z,697,Céline,Amaudruz,10852,20010080,Réforme de la direction de l'Etat,2012-03-14T12:50:01Z,NaN,Antrag der Minderheit Joder,Antrag der Mehrheit,7112,Loi sur l'organisation du gouvernement et de l'administration (LOGA),NaN,No,2796,UDC
1,2796,2020-09-04T05:45:44Z,697,Céline,Amaudruz,10853,20010080,Réforme de la direction de l'Etat,2012-03-14T12:51:52Z,NaN,NaN,NaN,7113,Loi sur l'organisation du gouvernement et de l'administration (LOGA),NaN,No,2796,UDC
2,2796,2020-09-04T05:45:44Z,697,Céline,Amaudruz,11498,20010080,Réforme de la direction de l'Etat,2012-09-10T15:56:03Z,NaN,Antrag der Minderheit Gross Andreas (Nichtabschreiben),Antrag der Mehrheit (Abschreiben),7758,Arrêté fédéral concernant la réforme de la direction de l'Etat,NaN,Yes,2796,UDC
3,2796,2020-09-04T05:45:44Z,697,Céline,Amaudruz,11500,20010080,Réforme de la direction de l'Etat,2012-09-10T15:58:33Z,NaN,Antrag der Minderheit Moret (Eintreten),Antrag der Mehrheit (Nichteintreten),7760,Arrêté fédéral sur la réforme du gouvernement,NaN,Yes,2796,UDC
4,2796,2020-09-04T05:45:44Z,697,Céline,Amaudruz,11583,20010080,Réforme de la direction de l'Etat,2012-09-12T08:32:11Z,NaN,Antrag der Minderheit Joder (streichen),Antrag der Mehrheit,7843,Loi sur l'organisation du gouvernement et de l'administration (LOGA),NaN,No,2796,UDC


# Rendons ça plus clair avec un tableau croisé dynamique, comme dans Excel

Ça s’appelle aussi une table pivot… dans Pandas: `.pivot_table()`


In [ ]:
dfm["timestamp"] = pd.to_datetime(dfm["date"])

In [ ]:
dfp = pd.pivot_table(
    dfm,
    values="decision",
    index=["affairId", "affairTitle", "timestamp"],
    aggfunc="sum",
    columns=["firstName", "lastName", "id", "party"],
)
dfp

firstName                                                                                                                                                    Andreas  \
lastName                                                                                                                                                        Aebi   
id                                                                                                                                                              2670   
party                                                                                                                                                            UDC   
affairId affairTitle                                                                                                               timestamp                           
20000431 Assurer l'encadrement législatif de l'activité de guide de montagne et du secteur des activités à risque                  2009-09-24 11:27:26+00:00      No   
                                                                                                                                   2009-09-24 11:33:19+00:00     Yes   
                                                                                                                                   2009-09-24 11:34:37+00:00      No   
                                                                                                                                   2010-12-17 10:01:09+00:00      No   
20000436 Prestations complémentaires pour des familles. Modèle tessinois                                                           2011-06-17 09:44:34+00:00     Yes   
...                                                                                                                                                              ...   
20110489 Abrogation de l’article 293 CP                                                                                            2017-03-15 11:25:35+00:00     NaN   
                                                                                                                                   2017-06-16 08:43:28+00:00     NaN   
20113767 Halte aux congés et aux sorties pour les personnes internées                                                              2016-06-14 12:53:14+00:00     NaN   
20120057 Développement de l’acquis de Schengen. Reprise du règlement portant création d’une agence pour des systèmes d‘information 2016-09-27 09:20:54+00:00     NaN   
                                                                                                                                   2016-09-27 09:22:24+00:00     NaN   

firstName                                                                                                                                                      Céline  \
lastName                                                                                                                                                     Amaudruz   
id                                                                                                                                                               2796   
party                                                                                                                                                             UDC   
affairId affairTitle                                                                                                               timestamp                            
20000431 Assurer l'encadrement législatif de l'activité de guide de montagne et du secteur des activités à risque                  2009-09-24 11:27:26+00:00      NaN   
                                                                                                                                   2009-09-24 11:33:19+00:00      NaN   
                                                                                                                                

Giga. On va enregistra ça en csv et en Excel.

Je ne peux pas vous garantir que toutes les opérations soient correctes, ni deviner quelles erreurs pourraient potentiellement se glisser dans les données du parlement: si vous en faites quelque chose, ce sera à vous de vérifier que tout soit correct et de contacter les personnes concernées pour leur donner la parole.


In [ ]:
dfp.to_csv("data/votes_cn.csv")

In [15]:
dfpx = dfp.reset_index()

In [ ]:
dfpx["timestamp"] = dfpx["timestamp"].astype(str)

In [ ]:
dfpx.to_excel("data/votes_cn.xlsx")

In [18]:
!open data/votes_cn.xlsx